# Вариант с SKLearn

## Тест нормального скрипта

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from re import split, search
import re

In [4]:
def fit_model(clf, train, **params):
    texts = train['Описание']
    texts_labels = train['Газоснабжение финал']
    model = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1, 3))), ('clf', clf(**params))])
    model.fit(texts, texts_labels)
    return model

In [5]:
def prepare(train):
    model = fit_model(SGDClassifier, train, **{'loss': 'hinge', 'penalty': 'none', 'learning_rate': 'constant', 'eta0': 0.5})
    return model

In [21]:
df = pd.read_excel('input.xlsx', sheet_name='газоснабжение', usecols=['Описание', 'Газоснабжение из описания', 'Газоснабжение финал'])
df = df.apply(clear, axis=1)
model = prepare(df)


C:\Users\v.komarnickiy\conda\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [22]:
# train = df.iloc[:int(len(df) * 0.8)]
# test  = df.iloc[int(len(df) * 0.8):]
test = pd.read_excel('test.xlsx', sheet_name='Лист1', usecols=['Уникальный идентификационный номер Ч', 'Описание'])
test = test.apply(clear, axis=1)
test['Predicted'] = [None] * len(test)
res = model.predict(test['Описание'])
test.loc[:, 'Predicted'] = res

TypeError: ('expected string or bytes-like object', 'occurred at index 8681')

In [19]:
test[test.Predicted != test['Газоснабжение финал']].to_excel('temp.xlsx')

In [20]:
len(test[test.Predicted != test['Газоснабжение финал']]) / len(test)

0.060073132509141564

In [6]:
import pandas as pd
from re import search

In [43]:
# Load excel into `df`
df = pd.read_excel('input.xlsx', sheet_name='газоснабжение', usecols=['Описание', 'Газоснабжение из описания', 'Газоснабжение финал'])

In [44]:
df['Predicted'] = [None] * len(df)

In [14]:
train = df.iloc[int(len(df) * 0.2):]
test = df.iloc[:int(len(df) * 0.2)]

### Оставляем только предложения с газом

In [14]:
def clear(row):
    try:
        regex = r'\bгаз|\bкоммуник'
        sentences = split(r'[.?!]', row.at['Описание'])
        results = []
        for s in sentences:
            if search(regex, s, flags=re.IGNORECASE):
                results.append(s)
        if not results:
            row.at['Predicted'] = 'нет'
            return row
        else:
            descr = ' '.join(results)
            splitted = list(filter(lambda x: x, split(r'[!?., ]', descr)))
            new_descr = ''
            if len(splitted) > 10:
                for i, e in enumerate(splitted):
                    if search(regex, e, re.IGNORECASE):
                        start = max(i - 10, 0)
                        end = min(i + 11, len(splitted))
                        new_descr = ' '.join(splitted[start: end])
                        break
            if new_descr:
                row.at['Описание'] = new_descr
            else:
                row.at['Описание'] = descr
    except Exception as e:
        print(f'{e.__class__.__name__}: {e}')
    finally:
        return row

In [48]:
filtered = df.apply(clear, axis=1)  #.loc[18, 'Описание']
filtered[filtered['Predicted'] != 'нет']

In [7]:
test.loc[:, 'Описание'] = test['Описание'].apply(clear).iloc[2]

In [8]:
train.loc[:, 'Описание'] = train['Описание'].apply(clear).iloc[2]


In [15]:
texts = train['Описание']
texts_labels = train['Газоснабжение финал']

## RandomForest

In [7]:
"""
Full text   - 12%
+-50        - 7%
"""

rf = Pipeline([
                     ('tfidf', TfidfVectorizer(ngram_range=(1, 3))),
                     ('clf', RandomForestClassifier())
                     ])
 
rf.fit(texts, texts_labels)


In [8]:
res = rf.predict(test['Описание'])

In [9]:
test.loc[:, 'Predicted'] = res

In [10]:
print(f"Ошибка RandomForest: {len(test[test['Predicted'] != test['Газоснабжение финал']])/len(test) * 100:.2f}%")

## SGDClassifier

In [11]:
sgd = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 3))),
    ('clf', SGDClassifier(loss='perceptron', penalty='l2', learning_rate='adaptive', eta0=0.001))
])

sgd.fit(texts, texts_labels)

In [12]:
"""
Full text:
hinge                       - 10%
log                         - 20%
modified_huber              - 8 %
squared_hinge               - 8 %
perceptron                  - 7 %
squared_loss                - 10%
huber                       - 27%
epsilon_insensitive         - 11%
squared_epsilon_insensitive - 80%

+-50:
perceptron                  - 27%
"""

res = sgd.predict(test['Описание'])
test.loc[:, 'Predicted'] = res
print(f"Ошибка SGD: {len(test[test['Predicted'] != test['Газоснабжение финал']])/len(test) * 100:.2f}%")

## Multinomial Naive Bayes

In [24]:
"""
+-50        - 27%
Full text   - 39%
"""

nb = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 3))),
    ('clf', MultinomialNB())
])

nb.fit(texts, texts_labels)

In [25]:
res = nb.predict(test['Описание'])
test.loc[:, 'Predicted'] = res
print(f"Ошибка NB: {len(test[test['Predicted'] != test['Газоснабжение финал']])/len(test) * 100:.2f}%")

## KNeighbours

In [26]:
"""
Full text       - 23%
+-50            - 81%
"""

knn = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 3))),
    ('clf', KNeighborsClassifier())
])

knn.fit(texts, texts_labels)

In [27]:
res = knn.predict(test['Описание'])
test.loc[:, 'Predicted'] = res
print(f"Ошибка KNN: {len(test[test['Predicted'] != test['Газоснабжение финал']])/len(test) * 100:.2f}%")

## Support Vectors

In [6]:
"""
+-50        - 
Full text   -   
"""

svc = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 3))),
    ('clf', SVC())
])

svc.fit(texts, texts_labels)

In [0]:
res = svc.predict(test['Описание'])
test.loc[:, 'Predicted'] = res
print(f"Ошибка GNB: {len(test[test['Predicted'] != test['Газоснабжение финал']])/len(test) * 100:.2f}%")

## Тест кучи классификаторов

In [8]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from itertools import product
import logging as log

In [9]:
log.basicConfig(filename='classifiers.log', format = '[LINE:%(lineno)d]# [%(asctime)s]  %(message)s', level = log.INFO)

In [10]:
classifiers = [
    # KNeighborsClassifier,
    # GaussianProcessClassifier,
    # DecisionTreeClassifier,
    # RandomForestClassifier,
    # MLPClassifier,
    # AdaBoostClassifier,
    # GaussianNB,
    # QuadraticDiscriminantAnalysis,
    # SVC
    SGDClassifier
]
params = {
    KNeighborsClassifier: {
        'n_neighbors': [3, 4, 5, 6, 7],
        'weights': ['uniform', 'distance'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
    },
    SVC: {
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']
    },
    GaussianProcessClassifier: {

    },
    DecisionTreeClassifier: {
        'criterion': ['gini', 'entropy']
    },
    RandomForestClassifier: {
        'criterion': ['gini', 'entropy'],
        'n_estimators': [10, 100]
    },
    MLPClassifier: {
        'activation': ['identity', 'logistic', 'tanh', 'relu'],
        'solver': ['lbfgs', 'sgd', 'adam'],
        'learning_rate': ['constant', 'invscaling', 'adaptive']
    },
    AdaBoostClassifier: {

    },
    GaussianNB: {

    },
    QuadraticDiscriminantAnalysis: {

    },
    SGDClassifier: {
        'loss': ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron', 'squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
        'penalty': ['none', 'l2', 'l1', 'elasticnet'],
        'learning_rate': ['constant', 'optimal', 'invscaling', 'adaptive'],
        'eta0': [0.1, 0.01, 0.001, 0.25]
    }
}

In [11]:
# knn - 17.6%
log.info('Started')
try:
    for clf in classifiers:
        kw_list = []
        for param in params[clf].items():
            kw_list.append(list(product([param[0]], param[1])))
        sets = map(lambda x: dict(x), product(*kw_list))
        for params_set in sets:
            model = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1, 3))), ('clf', clf(**params_set))])
            model.fit(texts, texts_labels)
            res = model.predict(test['Описание'])
            test.loc[:, 'Predicted'] = res
            log.info(f"Ошибка {clf.__name__}: {len(test[test['Predicted'] != test['Газоснабжение финал']])/len(test) * 100:.2f}% with params {params_set}")
        if not sets:
            model = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(1, 3))), ('clf', clf())])
            model.fit(texts, texts_labels)
            res = model.predict(test['Описание'])
            test.loc[:, 'Predicted'] = res
            log.info(f"Ошибка {clf.__name__}: {len(test[test['Predicted'] != test['Газоснабжение финал']])/len(test) * 100:.2f}% without params")
finally:
    log.info('Finished')

In [21]:
optimal_params = {'loss': 'hinge', 'penalty': 'none', 'learning_rate': 'constant', 'eta0': 0.5}
clf = SGDClassifier
model = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 3))),
    ('clf', clf(**optimal_params))
]).fit(texts, texts_labels)
res = model.predict(test['Описание'])
test.loc[:, 'Predicted'] = res

In [27]:
print(f"Ошибка {clf.__name__}: {len(test[test['Predicted'] != test['Газоснабжение финал']])/len(test) * 100:.2f}% with params {optimal_params}")
test